In [1]:
filepath = '/content/drive/MyDrive/Colab Notebooks/mnist.zip'

In [5]:
import zipfile
zipex = zipfile.ZipFile(filepath)
zipex.extractall('/content/drive/MyDrive/Colab Notebooks/mnist')
zipex.close()    

In [9]:
# !unzip -qq '/content/drive/MyDrive/Colab Notebooks/mnist.zip'

replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense

In [13]:
from scipy.sparse import random
# 데이터 로드
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mnist/train.csv',index_col=0)
x_train = train_df.iloc[:,1:]
y_train = train_df.iloc[:,0]
x_train = x_train.astype('float32')/255.  # 데이터 정규화
y_train = tf.keras.utils.to_categorical(y_train,10)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,random_state=32)
x_train.shape, y_train.shape

((45000, 784), (45000, 10))

In [14]:
inputs = tf.keras.Input(shape=(28,28,1))
x = Conv2D(32,3,activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = MaxPooling2D(2)(x)
x = Conv2D(16,3,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = MaxPooling2D(2)(x)
x = Conv2D(32,3,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = MaxPooling2D(2)(x)
x = Flatten()(x)
x = Dense(64,activation='relu')(x)
outputs = Dense(10,activation='softmax')(x)

In [16]:
model = tf.keras.Model(inputs,outputs)
callback = [
    tf.keras.callbacks.EarlyStopping(patience=5)
]
model.compile(loss='mse',
              optimizer='adam',metrics=['acc'])
x_train_reshape = x_train.to_numpy().reshape(-1,28,28,1)
x_test_reshape = x_test.to_numpy().reshape(-1,28,28,1)
history = model.fit(x_train_reshape,y_train, batch_size=126
                    , epochs=200,callbacks =callback ,validation_data=(x_test_reshape,y_test))

Epoch 1/200
358/358 [==============================] - 3s 8ms/step - loss: 0.0045 - acc: 0.9709 - val_loss: 0.0041 - val_acc: 0.9743
Epoch 2/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0032 - acc: 0.9794 - val_loss: 0.0034 - val_acc: 0.9780
Epoch 3/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0027 - acc: 0.9837 - val_loss: 0.0033 - val_acc: 0.9781
Epoch 4/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0022 - acc: 0.9861 - val_loss: 0.0028 - val_acc: 0.9825
Epoch 5/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0019 - acc: 0.9885 - val_loss: 0.0027 - val_acc: 0.9824
Epoch 6/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0017 - acc: 0.9895 - val_loss: 0.0031 - val_acc: 0.9800
Epoch 7/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0017 - acc: 0.9897 - val_loss: 0.0027 - val_acc: 0.9829
Epoch 8/200
358/358 [==============================] - 2s 6ms/step - 

In [36]:
# 모델이 완성되었으니  제공한 테스트를 돌려서 제출
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mnist/test.csv',index_col=0)
x_test = x_test.astype('float32')/255.  # 데이터 정규화

In [35]:
len(test_df)

10000

In [37]:
test_reshape = test_df.to_numpy().reshape(-1,28,28,1)
test_reshape.shape

(10000, 28, 28, 1)

In [38]:
result = model.predict(test_reshape)

313/313 [==============================] - 1s 2ms/step


In [39]:
len(np.argmax(result,axis=1))

10000

In [ ]:
[np.argmax(i) for i in result ]

In [41]:
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mnist/sample_submission.csv',index_col=0)

In [42]:
submission_df['label'] = [np.argmax(i) for i in result ]

In [45]:
submission_df.to_csv('submission.csv')